In [147]:
import requests
import json
import time
import numpy as np
import pandas as pd

In [3]:
def get_kegg_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response
    else:
        return f"Error: {response.status_code}"

# Step 1 : Obtain human metabolic pathways

In [161]:
pathways_url = "http://rest.kegg.jp/list/pathway/hsa"
response = get_kegg_url(pathways_url)

In [173]:
pathways = {}
for line in response.text.split('\n')[:-1]:
    code, name = line[:-23].split('\t')
    pathways[code] = name
pathways

{'hsa01100': 'Metabolic pathways',
 'hsa01200': 'Carbon metabolism',
 'hsa01210': '2-Oxocarboxylic acid metabolism',
 'hsa01212': 'Fatty acid metabolism',
 'hsa01230': 'Biosynthesis of amino acids',
 'hsa01232': 'Nucleotide metabolism',
 'hsa01250': 'Biosynthesis of nucleotide sugars',
 'hsa01240': 'Biosynthesis of cofactors',
 'hsa00010': 'Glycolysis / Gluconeogenesis',
 'hsa00020': 'Citrate cycle (TCA cycle)',
 'hsa00030': 'Pentose phosphate pathway',
 'hsa00040': 'Pentose and glucuronate interconversions',
 'hsa00051': 'Fructose and mannose metabolism',
 'hsa00052': 'Galactose metabolism',
 'hsa00053': 'Ascorbate and aldarate metabolism',
 'hsa00500': 'Starch and sucrose metabolism',
 'hsa00520': 'Amino sugar and nucleotide sugar metabolism',
 'hsa00620': 'Pyruvate metabolism',
 'hsa00630': 'Glyoxylate and dicarboxylate metabolism',
 'hsa00640': 'Propanoate metabolism',
 'hsa00650': 'Butanoate metabolism',
 'hsa00562': 'Inositol phosphate metabolism',
 'hsa00190': 'Oxidative phospho

In [180]:
# Keeping only metabolic pathways, not disease or signaling
def filter_pathways(pair):
    key, value = pair
    pid = key[3:]
    keeper = np.arange(0000, 1300, 1)
    if int(pid) in keeper:
        return True
    else:
        return False

pathways_m = dict(filter(filter_pathways, pathways.items()))

In [215]:
# Popping out the general pathway containing all metabolic genes
pathways_m.pop('hsa01100')
len(pathways_m)

92

In [216]:
with open("kegg_pathways.json", "w") as outfile: 
    json.dump(pathways, outfile)

# Step 2 : Obtain the list of gene associated with each pathway

## Get ncbi IDs from KEGG API

In [76]:
genes_url = f"http://rest.kegg.jp/link/ko/pathway"
genes = get_kegg_url(genes_url)

In [79]:
# Obtain NCBI gene IDs
genes_url = f"http://rest.kegg.jp/link/hsa/pathway"
genesh = get_kegg_url(genes_url)

In [81]:
genes_data = genesh.text.split('\n')

In [103]:
ncbi_ids = {}
for line in genes_data[:-1]:
    pathway, gene = line.split('\t')
    pathway = pathway[5:]
    if pathway in res:
        ncbi_ids[pathway].append(gene[4:])
    else :
        ncbi_ids[pathway] = [gene[4:]]

In [200]:
ncbi_ids

{'hsa00010': ['10327',
  '124',
  '125',
  '126',
  '127',
  '128',
  '130',
  '130589',
  '131',
  '160287',
  '1737',
  '1738',
  '2023',
  '2026',
  '2027',
  '217',
  '218',
  '219',
  '2203',
  '221',
  '222',
  '223',
  '224',
  '226',
  '229',
  '230',
  '2538',
  '2597',
  '26330',
  '2645',
  '2821',
  '3098',
  '3099',
  '3101',
  '387712',
  '3939',
  '3945',
  '3948',
  '441531',
  '501',
  '5105',
  '5106',
  '5160',
  '5161',
  '5162',
  '5211',
  '5213',
  '5214',
  '5223',
  '5224',
  '5230',
  '5232',
  '5236',
  '5313',
  '5315',
  '55276',
  '55902',
  '57818',
  '669',
  '7167',
  '80201',
  '83440',
  '84532',
  '8789',
  '92483',
  '92579',
  '9562'],
 'hsa00020': ['1431',
  '1737',
  '1738',
  '1743',
  '2271',
  '3417',
  '3418',
  '3419',
  '3420',
  '3421',
  '4190',
  '4191',
  '47',
  '48',
  '4967',
  '50',
  '5091',
  '5105',
  '5106',
  '5160',
  '5161',
  '5162',
  '55753',
  '6389',
  '6390',
  '6391',
  '6392',
  '8801',
  '8802',
  '8803'],
 'hsa00030

In [155]:
with open("ncbi_ids.json", "w") as outfile: 
    json.dump(ncbi_ids, outfile)

## Get Gene symbol (names, HGNC) from NCBI Entrez API

API requests are made with a 1 second delay as Entrez guideline advise not to make more than 3 requests per second.

In [117]:
def fetch_gene_symbols(genelist):
    api_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi'
    geneID_list = ','.join(genelist)
    url = f'{api_url}?db=gene&id={geneID_list}&retmode=json'
    response = requests.get(url)
    if response.status_code != 200:
        return f"Error: {response.status_code}"
    else:
        return response

In [129]:
def extract_symbols(json_data):
    symbol_list = []
    for uid in json_data['result']['uids']:
            gene_info = json_data['result'][uid]
            symbol_list.append(gene_info['nomenclaturesymbol'])
    return symbol_list

In [205]:
def get_official_symbols(ncbi_ids):
    genes_pathways = {}
    for pathway in ncbi_ids.keys() :
        time.sleep(1)
        ncbi_rep = fetch_gene_symbols(ncbi_ids[pathway])
        json_rep = ncbi_rep.json()
        symbols = extract_symbols(json_rep)
        genes_pathways[pathway] = symbols
        print(f'Pathway {pathway} : {ncbi_rep}')
    return genes_pathways

In [210]:
# Had to run this because initially the global pathway 'hsa01100' was not excluded from the list
# and it contains > 1500 genes which exceeds the NCBI limit of 200 per request and causes an error

missing_pathways = {}
for item in pathways_m.items():
    key, value = item
    if key not in genes_pathways:
        missing_pathways[key] = value

ncbi_ids_missing = {key:value for key, value in ncbi_ids.items() if key in missing_pathways}
ncbi_ids_missing.pop('hsa01100')
for key, value in ncbi_ids_missing.items():
    print(f'Pathway {key} : {len(value)} genes')


Pathway hsa01200 : 116 genes
Pathway hsa01210 : 33 genes
Pathway hsa01212 : 57 genes
Pathway hsa01230 : 75 genes
Pathway hsa01232 : 85 genes
Pathway hsa01240 : 153 genes
Pathway hsa01250 : 37 genes


In [211]:
genes_pathway_missing = get_official_symbols(ncbi_ids_missing)
len(genes_pathway_missing)

Pathway hsa01200 : <Response [200]>
Pathway hsa01210 : <Response [200]>
Pathway hsa01212 : <Response [200]>
Pathway hsa01230 : <Response [200]>
Pathway hsa01232 : <Response [200]>
Pathway hsa01240 : <Response [200]>
Pathway hsa01250 : <Response [200]>


7

In [214]:
print(len(genes_pathways))
genes_pathways_all = {**genes_pathways, **genes_pathway_missing}
print(len(genes_pathways_all))

85
92


In [218]:
with open("gene_symbols.json", "w") as outfile: 
    json.dump(genes_pathways, outfile)